## Workshop 3: This notebook analyzes data for pricing optimization from an A/B test targeting conversion rates for an app. 

### Three main questions are addressed:
1) At which price should the company list their software? <br>
2) What attributes of customer behavior influence conversion rate? <br>
3) How long should the experiment have taken? <br>

### The results are summarized below:
1) The company had a higher conversion rate for lower priced software. However, the difference in conversion rates did not outweigh the revenue generated by the higher priced software. Therefore, the company should list their software at a higher price.<br>
<br>
2) The top three sources of the highest conversion rates were: 1) Friend Referral,  2) Seo-bing, 3) Ads-google. The company could invest more in these domains given that they work and eliminate sources with low conversion rates, or rethink the strategy for sources with low conversion rates. In addition, windows and android users are less likely to convert, and a targeted marketing approach should be implemented towards these groups. <br>
<br>
3) Based on a power analysis, the experiment could have been conducted in 5 days with 34,578 users. However, I would recommend 2 weeks due to variations in user activity based on day of the week.

In [ ]:
# import necessary packages
import pandas as pd
import numpy as np
#import pandas_profiling
import seaborn as sns
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [ ]:
# read in data
test_results = pd.read_csv("test_results.csv")
user_table = pd.read_csv("user_table.csv")

### Check data for any abnormalities or missing values

In [ ]:
# inspect the test data
test_results = test_results.drop("Unnamed: 0", axis=1)
test_results.head()

In [ ]:
# inspect the users table
user_table.head()

In [ ]:
# inspect data types for test results
print("Entries in test results table:", test_results.shape[0])
print("")
test_results.info()

316800 total entries. There are missing values for the timestamp, source, and operative system variables.

In [ ]:
# number of null values
print("timestamp null values is", test_results.timestamp.isnull().sum())
print("source null values is", test_results.source.isnull().sum())
print("OS null values is", test_results.operative_system.isnull().sum())

In [ ]:
# inspect data types for user table
print("Entries in user table:", user_table.shape[0])
print("")
user_table.info()

All data are present for the user table, but there is a mismatch between the number of <br>
users in this table and the number of users in test results. More users in the test <br>
results table.

In [ ]:
# are there duplicate users in the test results table? --- No, just more users 
print("Number of unique user ids is:", test_results.user_id.nunique())

There are more users in the test results table than in the users table, and all users in the test results table are unique. This means that not every user in the test results table will appear in the user table.

## Explore and Clean Data 

I will use the pandas profiling tool to get a snapshot of my data. This is an awesome tool for understanding the distribution of each variable and for finding anything weird in the data set.

In [ ]:
# pandas_profiling.ProfileReport(test_results)

From the test results profile: <br>
- I see that most users did not convert. This makes an imbalanced class issue if I attempt to model the data to answer the second question. 
- Operative system has both missing values and inconsistent data formatting that will need to be cleaned up.
- Price has 3076 distinct values when it should only have two. 
- Source is missing data.
- Timestamp has missing data.

In [ ]:
# inspect operative systems values
test_results.operative_system.value_counts()

In [ ]:
# fix operative systems values
test_results["operative_system"] = test_results.operative_system.replace({"i0S": "iOS", "wnddows": "windows",
                                   "anroid":"android", "apple":"mac"})

Now, the number of unique values for price should be 2 (39 or 59), but getting 3000+ values.

In [ ]:
# inspect number of unique levels of price variable
test_results.price.nunique()

I am choosing to drop the price column for the dataframe. This is because the price column is the equivalent to the test column where 0=39 and 1=59, so it is redundant. Dropping solves the issue.

In [ ]:
# drop price column
test_results = test_results.drop("price", axis=1)

The source column is missing values. I’m not sure why this is, so I am going to keep those rows and simply indicate that they are missing data.

In [ ]:
test_results["source"] = np.where(test_results.source.isnull(), "missing", test_results.source)

### Q1: At which price should the company list their software?

Let’s take a look at how many users converted in each group (39 and  59).

In [ ]:
# examine number of converteds for each group
test_results.groupby(["test"])["user_id"].count()

More converted when the price was 39 compared to 59, but the sample size in each group was unequal. So, this alone does not mean $39 is better. There is a cost-benefit trade-off to examine and we need to figure out the conversion rate first.

To start, for each pricing group, what proportion of that group bought the software? This is the CONVERSION RATE.

In [ ]:
# find the conversion rate
conv = test_results.groupby("test").converted.value_counts(normalize=True)
conv

In [ ]:
print("The conversion rate for the $39 group is:", round(conv[0,1]*100,2), "%")
print("The conversion rate for the $59 group is:", round(conv[1,1]*100,2), "%")

In [ ]:
test_results.groupby(["test"])["converted"].value_counts(normalize=True).unstack(
    'converted').plot(kind="bar", stacked=True, figsize=(5,4));
plt.xticks(np.arange(2), ["$39","$59"],rotation=0)
plt.title("Proportion of conversions in each group");

The less expensive software is more likely to sell, but this might not matter when considering the difference in revenue. <br>

In [ ]:
How much money would the company make from 1 user at each price?

In [ ]:
How much money would the company make from 1 user at each price

In [ ]:
How much money would the company make from 1 user at each price

In [ ]:
How much money would the company make from 1 user at each price

In [ ]:
# how much money is made? estimated revenue per 1 user
print("Estimated revenue per 1 user made by $39:", 39*.0199) 
print("Estimated revenue per 1 user made by $59:", 59*.0155) 

Are the proportions of converted in each group statistically different?

In [ ]:
val_counts = test_results.groupby("test")["converted"].value_counts()
val_counts

In [ ]:
# This uses a simple normal test for proportions. It should be the
# same as running the mean z-test on the data encoded 
# 1 for event and 0 for no event so that the sum corresponds to the count.

from statsmodels.stats.proportion import proportions_ztest

count = np.array([val_counts[0,1],val_counts[1,1]])
nobs = np.array([val_counts[0,0]+val_counts[0,1],val_counts[1,0]+val_counts[1,1]])
value = .05
stat, pval = proportions_ztest(count, nobs, value)
print("pvalue is", pval)
print("statistically significant")

There is a significant difference in conversion rates found via a z-test.

The drop in conversion is not significant enough to continue with the lower priced software. More money will be made selling the higher priced software, even though the conversion rate is lower. I suggest the company implement the higher priced software to increase revenue.

## Q2: What attributes of customer behavior influence conversion rate?

In [ ]:
# merge user table with results tables
# left merge because there are extra users in the test_results table
merged_df = pd.merge(test_results, user_table, on="user_id", how="left")

In [ ]:
merged_df.describe(include="all")

In [ ]:
# just select meaningful columns for analysis
cols = ["user_id", "timestamp", "source", "device", "operative_system", "test", "converted", "city"]
merged_df = merged_df[cols]

In [ ]:
merged_df.head()

What was the conversion rate for each source?

In [ ]:
merged_df.groupby(["source"])["converted"].value_counts(normalize=True).sort_values(ascending=False).unstack('converted').plot(kind='bar', stacked=True, figsize=(8,4));
plt.legend(loc='upper center', bbox_to_anchor=(1.03, 1))
plt.xticks(rotation=45)
plt.title("Proportion of conversions by source");

In [ ]:
src = merged_df.groupby(["source"])["converted"].value_counts(normalize=True).sort_values(ascending=False)

In [ ]:
# display conversion rate for each source
print("Ads-bing conversion rate is:", round(src["ads-bing",1]*100,2))
print("Ads-google conversion rate is:", round(src["ads-google",1]*100,2))
print("Ads-yahoo conversion rate is:", round(src["ads-yahoo",1]*100,2))
print("Ads-facebook conversion rate is:", round(src["ads_facebook",1]*100,2))
print("Ads-other conversion rate is:", round(src["ads_other",1]*100,2))
print("Direct traffic conversion rate is:", round(src["direct_traffic",1]*100,2))
print("Friend referral conversion rate is:", round(src["friend_referral",1]*100,2))
print("Seo-bing conversion rate is:", round(src["seo-bing",1]*100,2))
print("Seo-google conversion rate is:", round(src["seo-google",1]*100,2))
print("Seo-other conversion rate is:", round(src["seo-other",1]*100,2))
print("Seo-yahoo conversion rate is:", round(src["seo-yahoo",1]*100,2))
print("Seo-facebook conversion rate is:", round(src["seo_facebook",1]*100,2))
print("Missing conversion rate is:", round(src["missing",1]*100,2))

The top three highest conversion rates are: 1) Friend Referral, 2) Seo-bing, 3) Ads-google

For each device, what proportion bought the software? This is the conversion rate/device.

In [ ]:
merged_df.groupby(["device"])["converted"].value_counts(normalize=True).unstack('converted').plot(kind='bar', stacked=True, figsize=(7,5));
plt.xticks(rotation=0);
plt.title("Proportion of converteds in device");

Conversion rates are roughly equal between devices.

What kinds of operating systems are users using? Does the conversion rate differ depending on the OS?

In [ ]:
merged_df.groupby(["operative_system"])["converted"].value_counts(normalize=True).unstack('converted').plot(kind='bar', stacked=True, figsize=(7,4));
plt.title("Proportion of conversions by OS")
plt.legend(loc='upper center', bbox_to_anchor=(1.03, 1));

In [ ]:
os_ = merged_df.groupby(["operative_system"])["converted"].value_counts(normalize=True)

In [ ]:
# display conversion rate for each device
print("Android conversion rate is:", round(os_["android",1]*100,2))
print("iOS conversion rate is:", round(os_["iOS",1]*100,2))
print("Mac conversion rate is:", round(os_["mac",1]*100,2))
print("Windows conversion rate is:", round(os_["windows",1]*100,2))

iOS and mac are quite a bit higher, why? Could target windows and android users more.

### Q3: How long should the test have been run for to achieve statistically significant results?

This likely depends on how many users were visiting the site each day - power analysis, sample size estimation. <br>
First calculate the sample size needed:

In [ ]:
import statsmodels.stats.api as sms

# effect size for a test comparing two proportions
es = sms.proportion_effectsize(0.0155, 0.0199)

# solve for sample size parameter of the power of a two sample z-test
sample_ = sms.NormalIndPower().solve_power(es, power=0.8, alpha=0.05, ratio=1)
print("Need a sample size per group of:", round(sample_,0))
print("Need total user sample of:", round(sample_,0)*2)

Now calculate the number of users per day to estimate the total number of days needed

In [ ]:
# change to datetime
merged_df["timestamp"] = pd.to_datetime(merged_df.timestamp,format="%Y-%m-%d %H:%M:%S",errors='coerce')

In [ ]:
# now calculate users per days
t = merged_df.set_index("timestamp")
users_per_day = t.resample("D").user_id.count().mean()
print("Average users per day:", round(users_per_day,2))

In [ ]:
print("Total time needed for experiment is:", round(sample_/users_per_day,2), "days")

The total time needed for the experiment is 5 days, but I would recommend longer 2 weeks due to variations in user activity based on day of the week.